In [37]:
import xarray as xr
import os
import re
from datetime import date 


# Path to the folder with the daily data:
folder_path = r"../data/raw/daily_Windsat"

# Temporary path for the reduced data
reduced_path = r"../data/processed/temporary"

os.makedirs(reduced_path)

In [38]:
# List all the file names in the folder, save the date and append it to a list:

file_paths = []
day_numbers = [] # Day number of 2017
for file in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, file)):
        file_paths.append(os.path.join(folder_path, file))

        # Regex to extract date
        ymd_regex = r"_(\d{4})_(\d{2})_(\d{2})"

        year, month, day = [int(number) for number in re.findall(ymd_regex, file)[0]]
        day_numbers.append(date(year, month, day).timetuple().tm_yday)

# Sort file paths and dates based on dates
file_paths_dates_sorted = sorted(zip(file_paths, day_numbers), key=lambda x: x[1])

In [39]:
def select_datavars(dataset: xr.Dataset) -> xr.Dataset:
    """ 
        Select only relevant variables (for Brightness temperature)
    """

    selected_dvars = [
        "longitude",
        "latitude",
        "node", # node of swath
        # "look", # look direction. we will select only look = 0 (forward)
        "frequency_vpol", # center frequency of V-pol channel in each band
        "frequency_hpol", # center frequency of H-pol channel in each band
        "eia_nominal", # nominal Earth indidence angle of each band
        "time", # Time of observation (lat, lon) seconds since 01 JAN 2000 00Z
        "eaa", # boresight Earth azimuth angle. range: [0o, 360o]. 
        "eia", #  boresight Earth incidence angle. range: [0o, 90o]   
        "tbtoa", # Brightness temperature
        "quality_flag", # 32-bit quality control flag
  
        # "sss_HYCOM", # HYCOM sea surface salinity
        # "surtep_REY", # NOAA (Reynolds) V2 OI sea surface temperature

        # # Land fractions
        # "fland_06", # for 6GHz
        # "fland_10", # For 10 GHz

        # # Windsat V8 products
        # "surtep_WSAT", # skin temperature
        # "colvap_WSAT", # atmosphere_mass_content_of_water_vapor
        # "colcld_WSAT", # atmosphere_mass_content_of_cloud_liquid_water
        # "winspd_WSAT", # sea surface wind speed
        # "rain_WSAT", # surface rain rate

        # # Cross-Calibrated Multi-Platform 
        # "winspd_CCMP", # Wind speed
        # "windir_CCMP", # Cross-Calibrated Multi-Platform Wind direction

        # # ERA 5 products
        # "surtep_ERA5", # skin temperature
        # "airtep_ERA5", # Air temperature at 2m above surface
        # "colvap_ERA5", # Columnar liquid cloud water
        # "colcld_ERA5", # atmosphere_mass_content_of_cloud_liquid_water
        # "winspd_ERA5", # 10-m NS wind speed
        # "windir_ERA5", # Wind direction

        # "surtep_CMC", # CMC Sea surface temperature
        # "rain_IMERG", # IMERG V6 surface rain rate

        # # RSS 2022 absorption model
        # "tran", # Total atmospheric transmittance computed from ERA atmospheric profiles and WSAT columnar vapor and cloud water 
        # "tbdw", # Atmospheric downwelling brightness temperature computed from ERA atmospheric profiles and WSAT columnar vapor and cloud water
    ]

    dataset = dataset[selected_dvars]

    return dataset

def select_dims(dataset: xr.Dataset) -> xr.Dataset:
    """ 
        Remove unused frequencies and polarizations

        Frequencies: 
        0 -- 6.8 GHz
        1 -- 10.7 GHz
        2 -- 18.7 GHz (Ku)
        3 -- 23.8 GHz
        4 -- 37.0 GHz (Ka)

        Polarizations: 
        0 -- V
        1 -- H
        (except for 6.8 and 23.8 GHz): 
        2 -- P (+45º)
        3 -- M (-45º)
        4 -- L (Circular Left)
        5 -- R (Circular Right)
    """

    # Select dimensions
    dataset = dataset.sel(
        indexers={
            "polarization": [0, 1],  # [ V, H ]
            "frequency_band": [2, 4],  # [ 18.7 GHz (Ku) , 37.0 GHz (Ka) ]
            "look_direction": 0,  # Forward
        }
    )
    return dataset

def transform_dataset(dataset: xr.Dataset) -> xr.Dataset:
    """ 
    Other transformations
    """
    # Roll lattitude grid so we have -180, 180 range
    dataset = dataset.roll(shifts={"longitude_grid": 4 * 180})
    # dataset["longitude_grid"].__setattr__("long_name", "Shifted to -180, 180 range")
    # dataset["longitude_grid"].__setattr__("valid_min", "-180")
    # dataset["longitude_grid"].__setattr__("valid_max", "180")

    # Extract latitude and longitude grid dimensions
    dataset = dataset.assign_coords(lat=dataset.latitude, lon=dataset.longitude)

    return dataset


In [7]:
# # Create an empty list to store the datasets
# datasets = []

# # Load each file into xarray dataset
# for file, _ in file_paths_dates_sorted[:5]:
#     try:
#         dataset = xr.open_dataset(file, decode_times=False)

#         dataset = select_datavars(dataset)
#         dataset = select_dims(dataset)
#         dataset = transform_dataset(dataset)

#         datasets.append(dataset)
#     except Exception as e:
#         print(f"Error loading file {file}: {e}")

In [8]:
# Concatenate datasets along a new time dimension
# combined_dataset = xr.concat(datasets, dim="date")

In [9]:
# # Add time coordinate based on the parsed dates
# combined_dataset["day_number"] = day_numbers[:5]
# combined_dataset["day_number"].attrs = {
#     "description": "Int: Day fo the Year 2017"
# }

In [10]:
# Save the combined dataset to a single file
# combined_dataset.to_netcdf("../data/processed/combined_Windsat.nc")

In [11]:
# ds = xr.open_dataset("../data/processed/combined_Windsat.nc", decode_times=False)
# ds

In [12]:
# ds.sel(date = 2)["tbtoa"].plot()

In [13]:
# ds.tbtoa.sel(date = 2, frequency_band = 0, polarization = 0, swath_sector= 0).plot()

# New approach: 
Select the data and save each array into a single file
keep the file names and the day numbers loaded

combine the datasets while they are not loaded in memory, use the retrieved dates to concatenate them.



In [40]:
#Path to the reduced data

# Load each file into xarray dataset
for file, _ in file_paths_dates_sorted:
    try:
        dataset = xr.open_dataset(file, decode_times=False)

        dataset = select_datavars(dataset)
        dataset = select_dims(dataset)
        dataset = transform_dataset(dataset)

        # Save the reduced dataset
        filename = file.split("\\")[-1]
        save_path = os.path.join(reduced_path, filename)
        dataset.to_netcdf(path = save_path)
        print(f"Reduced dataset saved in {save_path}.")

    except Exception as e:
        print(f"Error loading file {file}: {e}")

In [22]:
file_path =[file for file, _ in file_paths_dates_sorted][0]
print(file_path)

xr.open_dataset(file_path, decode_times=False)

../data/raw/daily_Windsat\RSS_WINDSAT_DAILY_TBTOA_MAPS_2017_01_01.nc


<xarray.Dataset> Size: 1GB
Dimensions:         (longitude_grid: 1440, latitude_grid: 720, swath_sector: 2,
                     look_direction: 2, frequency_band: 5, polarization: 4,
                     polarization_dual: 2)
Dimensions without coordinates: longitude_grid, latitude_grid, swath_sector,
                                look_direction, frequency_band, polarization,
                                polarization_dual
Data variables: (12/34)
    longitude       (longitude_grid) float32 6kB ...
    latitude        (latitude_grid) float32 3kB ...
    node            (swath_sector) int32 8B ...
    look            (look_direction) int32 8B ...
    frequency_vpol  (frequency_band) float32 20B ...
    frequency_hpol  (frequency_band) float32 20B ...
    ...              ...
    colcld_WSAT     (latitude_grid, longitude_grid, swath_sector) float32 8MB ...
    rain_IMERG      (latitude_grid, longitude_grid, swath_sector) float32 8MB ...
    rain_WSAT       (latitude_grid, longitude_grid, swath_sector) float32 8MB ...
    tran            (polarization_dual, frequency_band, latitude_grid, longitude_grid, look_direction, swath_sector) float32 166MB ...
    tbup            (polarization_dual, frequency_band, latitude_grid, longitude_grid, look_direction, swath_sector) float32 166MB ...
    tbdw            (polarization_dual, frequency_band, latitude_grid, longitude_grid, look_direction, swath_sector) float32 166MB ...
Attributes: (12/72)
    Conventions:                            CF-1.7
    title:                                  RSS WindSat TOA Brightness Temper...
    version:                                V01.0
    summary:                                The dataset contains RSS WindSat ...
    references:                              [1] T. Meissner et al., Remote S...
    acknowledgement:                        Funded under NASA Grant 80NSSC21K...
    ...                                     ...
    Source_of_ancillary_IMERG_rain_rate:    Huffman, G. et al.,  2019. NASA G...
    Source_of_ancillary_CCMP_wind:          Mears, C. et al., 2023.Remote Sen...
    Source_of_ancillary_ERA5:               ECMWF Reanalysis v5 (ERA5). https...
    Source_of_RSS_WindSat_AS_ECV:           https://www.remss.com/missions/wi...
    Source_of_ancillary_HYCOM_SSS:          Hybrid Coordinate Ocean Model, Gl...
    Source_of_WindSat_L1C_TB:               T. Meissner et al., Remote Sensin...

In [30]:
# Now, open the files and concatenated them without loading the data:
reduced_paths = []
day_numbers = [] # Day number of 2017
for file in os.listdir(reduced_path):
    if os.path.isfile(os.path.join(reduced_path, file)):
        reduced_paths.append(os.path.join(reduced_path, file))

        # Regex to extract date
        ymd_regex = r"_(\d{4})_(\d{2})_(\d{2})"

        year, month, day = [int(number) for number in re.findall(ymd_regex, file)[0]]
        day_numbers.append(date(year, month, day).timetuple().tm_yday)

# Sort file paths and dates based on dates
reduced_paths_dates_sorted = sorted(zip(file_paths, day_numbers), key=lambda x: x[1])

datasets = [xr.open_dataset(file_path, decode_times=False) for file_path, _ in reduced_paths_dates_sorted]

combined_dataset = xr.concat(datasets, dim = "day_number",data_vars=["tbtoa","quality_flag", "time", "eaa", "eia"])


combined_dataset["day_number"] = day_numbers
combined_dataset["day_number"].attrs = {
    "description": "Int: Day fo the Year 2017"
}

In [36]:
# Save the datacube into a single file:
combined_dataset.to_netcdf(path = r"../data/processed/Windsat2017_datacube.nc")

# Close the datasets once the process is finished
for dataset in datasets:
    dataset.close() 
print("Datasets closed")

<generator object <genexpr> at 0x000002130ACD0B30>

In [35]:
# Remove the temporary files:
files = os.listdir(reduced_path)

for f in files:
    file_path = os.path.join(reduced_path,f)
    try:
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Removed temporary file {file_path}")

    except Exception as e:
         print(f"Error removing file: {file_path}\n{e}")

Removed temporary file ../data/processed/reduced_windsat\RSS_WINDSAT_DAILY_TBTOA_MAPS_2017_01_01.nc
Removed temporary file ../data/processed/reduced_windsat\RSS_WINDSAT_DAILY_TBTOA_MAPS_2017_01_02.nc
Removed temporary file ../data/processed/reduced_windsat\RSS_WINDSAT_DAILY_TBTOA_MAPS_2017_01_03.nc
Removed temporary file ../data/processed/reduced_windsat\RSS_WINDSAT_DAILY_TBTOA_MAPS_2017_01_04.nc
Removed temporary file ../data/processed/reduced_windsat\RSS_WINDSAT_DAILY_TBTOA_MAPS_2017_01_05.nc
